
### SQLAlchemy Object Relational Tutorial

The `ORM` is constructed upon `SQLAlchemy Expression Language`.
The `ORM` is in contrast to the `SQLAlchemy Expression Language`.

* associating user-defined Python `classes` with database `tables`
* associating `instances` of those classes (objects) with `rows` in their corresponding tables

orm: metadata/engine/session




engine = create_engine('sqlite:///:memory:')

meta = MetaData()

employees = Table('employees', meta,
    Column('employee_id', Integer, primary_key=True),
    Column('employee_name', String(60), nullable=False, key='name'),
    Column('employee_dept', Integer, ForeignKey("departments.department_id"))
)
SQLemployees.create(engine)


employees.create(engine, checkfirst=True)
employees.drop(engine, checkfirst=False)

从table到class，通过mapper实现
从class到table，通过`class.__table__`得到

metadata.create_all(engine)
metadata.drop_all(engine)

DDL
DML
DCL


Version Check

In [47]:
# -*- coding: utf-8 -*-
import logging
import akshare as ak
import sqlalchemy as db  # Version Check
from sqlalchemy import create_engine  # Connecting
from sqlalchemy import MetaData, Table, Sequence, Column, Integer, String, MetaData, ForeignKey  # Define and Create Tables
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
import pandas as pd

# logging.basicConfig()函数是一个一次性的简单配置工具使
logging.basicConfig(**{
    'level': logging.INFO,
    'format': "%(asctime)s - %(levelname)s - %(message)s",
    'datefmt': "%m/%d/%Y %H:%M:%S %p"
})

logging.info(db.__version__)


logging.info('数据库引擎engine和逻辑元数据metadata独立，可以在任意阶段绑定')

# engine
logging.info('Create independent engine.')
connection_uri = "mysql+pymysql://{user}:{password}@{host}:{port}/{dbname}".format(**{
    "user": "root",
    "password": "mysql123456",
    "host": "localhost",
    "port": "3306",
    "dbname": "xinguan"
})
engine = db.create_engine(connection_uri)

# metadata
logging.info('Declare independent metadata.')
Base = declarative_base()  # for object
metadata = MetaData()  # for sqlexpression

logging.info('Model District.')
class District(Base):
    __tablename__ = 'district'
    __table_args__ = {'extend_existing': True}
    id = Column(Integer, Sequence('district_id_seq'), primary_key=True)
    province = Column(String(32))
    city = Column(String(32))
    district = Column(String(32))

    def __repr__(self):
        return '<District(province={province};city={city};district={district})>'.format(**{
            'province': self.province,
            'city': self.city,
            'district': self.district
        })
    
class DistrictTest(Base):
    __tablename__ = 'district_test'
    __table_args__ = {'extend_existing': True}
    id = Column(Integer, Sequence('district_test_id_seq'), primary_key=True)
    province = Column(String(32))
    city = Column(String(32))
    district = Column(String(32))

    def __repr__(self):
        return '<District(province={province};city={city};district={district})>'.format(**{
            'province': self.province,
            'city': self.city,
            'district': self.district
        })

logging.info('Model DistrictInfo.')
class DistrictInfo(Base):
    __tablename__ = 'district_info'
    __table_args__ = {'extend_existing': True}
    id = Column(Integer, Sequence('district_info_id_seq'), primary_key=True)
    province = Column(String(32))
    city = Column(String(32))
    district = Column(String(32))
    show_address = Column(String(64))
    full_address = Column(String(64))
    cnt_sum_certain = Column(Integer)  # -1: 表示有确诊但是确诊人数不详
    
class DistrictInfoTest(Base):
    __tablename__ = 'district_info_test'
    __table_args__ = {'extend_existing': True}
    id = Column(Integer, Sequence('district_info_test_id_seq',0,1), primary_key=True, autoincrement=True)
    province = Column(String(32))
    city = Column(String(32))
    district = Column(String(32))
    show_address = Column(String(64))
    full_address = Column(String(64))
    cnt_sum_certain = Column(Integer)  # -1: 表示有确诊但是确诊人数不详
    



# District.__table__.drop(engine, checkfirst=True)  
# DistrictInfo.__table__.drop(engine, checkfirst=True)

class Test(Base):
    __tablename__ = 'test'
    __table_args__ = {'extend_existing': True}
    id = Column(Integer, Sequence('test_id_seq'), primary_key=True, autoincrement=True)
    province = Column(String(32))
    city = Column(String(32))
    district = Column(String(32))

    def __repr__(self):
        return '<District(province={province};city={city};district={district})>'.format(**{
            'province': self.province,
            'city': self.city,
            'district': self.district
        })

Base.metadata.drop_all(engine)
Base.metadata.create_all(engine)


# session
logging.info('Handle session, bind session and engine.')
# he ORM’s “handle” to the database is the Session.
# we define a Session class which will serve as a factory for new Session objects
Session = sessionmaker(bind=engine)  # Factory
# Session = sessionmaker()
# Session.configure(bind=engine)
session = Session()
# session和engin绑定后，在session中调用metadata则无需再绑定engin

07/04/2020 17:39:18 PM - INFO - 1.2.15
07/04/2020 17:39:18 PM - INFO - 数据库引擎engine和逻辑元数据metadata独立，可以在任意阶段绑定
07/04/2020 17:39:18 PM - INFO - Create independent engine.
07/04/2020 17:39:18 PM - INFO - Declare independent metadata.
07/04/2020 17:39:18 PM - INFO - Model District.
07/04/2020 17:39:18 PM - INFO - Model DistrictInfo.
07/04/2020 17:39:18 PM - INFO - Handle session, bind session and engine.


In [53]:
dfs = []
covid_19_area_search_df = ak.covid_19_area_search(province="浙江省", city="宁波市", district="鄞州区")
dfs.append(covid_19_area_search_df)
print(covid_19_area_search_df)
covid_19_area_search_df = ak.covid_19_area_search(province="四川省", city="成都市", district="高新区")
dfs.append(covid_19_area_search_df)
print(covid_19_area_search_df)

DistrictInfoTest.__table__.drop(engine)
DistrictInfoTest.__table__.create(engine)

df = pd.concat(dfs)
df.to_sql(
    DistrictInfoTest.__tablename__, 
    con=engine, 
    if_exists='replace', 
    index=False
)

  province city district show_address          full_address cnt_sum_certain
0      浙江省  宁波市      鄞州区       都市森林二期  浙江省宁波市鄞州区钟公庙街道都市森林二期               1
1      浙江省  宁波市      鄞州区         演武小区     浙江省宁波市鄞州区白鹤街道演武小区               2
2      浙江省  宁波市      鄞州区         曙光大地     浙江省宁波市鄞州区东胜街道曙光大地               1
3      浙江省  宁波市      鄞州区         风格南岸     浙江省宁波市鄞州区首南街道风格南岸               1
4      浙江省  宁波市      鄞州区          幸福苑      浙江省宁波市鄞州区东柳街道幸福苑               1
5      浙江省  宁波市      鄞州区         盛世天城     浙江省宁波市鄞州区中河街道盛世天城               1
6      浙江省  宁波市      鄞州区       金桥水岸花园   浙江省宁波市鄞州区潘火街道金桥水岸花园               1
7      浙江省  宁波市      鄞州区       中海国际四期   浙江省宁波市鄞州区下应街道中海国际四期               2
8      浙江省  宁波市      鄞州区         河头佳苑      浙江省宁波市鄞州区横溪镇河头佳苑               2
9      浙江省  宁波市      鄞州区         张东周村      浙江省宁波市鄞州区瞻岐镇张东周村               1
   province city district show_address      full_address cnt_sum_certain
0       四川省  成都市      高新区       时代锦城酒店   四川省成都市高新区时代锦城酒店              -1
1       四川省  成都市  

In [50]:
df['id'] = None

In [52]:
df.to_sql(
    DistrictInfoTest.__tablename__, 
    con=engine, 
    if_exists='replace', 
    index=True
)

In [27]:
import akshare as ak
covid_19_csse_daily_df = ak.covid_19_csse_daily(date="2020-04-06")
print(covid_19_csse_daily_df)




         FIPS      Admin2  Province_State      Country_Region   Last_Update  \
0     45001.0   Abbeville  South Carolina                  US  4/6/20 23:22   
1     22001.0      Acadia       Louisiana                  US  4/6/20 23:22   
2     51001.0    Accomack        Virginia                  US  4/6/20 23:22   
3     16001.0         Ada           Idaho                  US  4/6/20 23:22   
4     19001.0       Adair            Iowa                  US  4/6/20 23:22   
...       ...         ...             ...                 ...           ...   
2803      NaN         NaN             NaN  West Bank and Gaza  4/6/20 23:21   
2804      NaN         NaN             NaN      Western Sahara  4/6/20 23:21   
2805      NaN         NaN             NaN              Zambia  4/6/20 23:21   
2806      NaN         NaN             NaN            Zimbabwe  4/6/20 23:21   
2807      NaN  unassigned         Wyoming                  US  4/6/20 23:22   

            Lat       Long_  Confirmed  Deaths  Rec

In [28]:
print(covid_19_csse_daily_df)

class DailyReport(Base):
    __tablename__ = 'daily_report'
    __table_args__ = {'extend_existing': True}
    id = Column(Integer, Sequence('daily_report_id_seq'), primary_key=True)
    province = Column(String(32))
    city = Column(String(32))
    district = Column(String(32))
    show_address = Column(String(64))
    full_address = Column(String(64))
    cnt_sum_certain = Column(Integer)  # -1: 表示有确诊但是确诊人数不详

         FIPS      Admin2  Province_State      Country_Region   Last_Update  \
0     45001.0   Abbeville  South Carolina                  US  4/6/20 23:22   
1     22001.0      Acadia       Louisiana                  US  4/6/20 23:22   
2     51001.0    Accomack        Virginia                  US  4/6/20 23:22   
3     16001.0         Ada           Idaho                  US  4/6/20 23:22   
4     19001.0       Adair            Iowa                  US  4/6/20 23:22   
...       ...         ...             ...                 ...           ...   
2803      NaN         NaN             NaN  West Bank and Gaza  4/6/20 23:21   
2804      NaN         NaN             NaN      Western Sahara  4/6/20 23:21   
2805      NaN         NaN             NaN              Zambia  4/6/20 23:21   
2806      NaN         NaN             NaN            Zimbabwe  4/6/20 23:21   
2807      NaN  unassigned         Wyoming                  US  4/6/20 23:22   

            Lat       Long_  Confirmed  Deaths  Rec

In [35]:

    
Test.__table__.drop(engine)
Test.__table__.create(engine)
covid_19_area_all_df = ak.covid_19_area_all()
covid_19_area_all_df.to_sql(
    Test.__tablename__,
    con=engine,
    if_exists='replace',
    index=True
)

In [54]:
conn = engine.connect()


In [56]:
list(conn.execute("show databases;"))

[('information_schema',),
 ('material',),
 ('mysql',),
 ('performance_schema',),
 ('rmis',),
 ('sakila',),
 ('test',),
 ('xinguan',)]

In [59]:
conn.execute("use rmis;")
list(conn.execute("show tables;"))

[('auth_group',),
 ('auth_group_permissions',),
 ('auth_permission',),
 ('django_admin_log',),
 ('django_content_type',),
 ('django_migrations',),
 ('django_session',),
 ('good_good',),
 ('good_goodmultimedia',),
 ('guardian_groupobjectpermission',),
 ('guardian_userobjectpermission',),
 ('invoice_expense',),
 ('invoice_goodcomment',),
 ('invoice_goodinpurchase',),
 ('invoice_goodinsale',),
 ('invoice_purchase',),
 ('invoice_refund',),
 ('invoice_sale',),
 ('invoice_salecomment',),
 ('label_barcode',),
 ('label_label',),
 ('label_labelseries',),
 ('label_labelstickgood',),
 ('label_labeltarget',),
 ('organization_store',),
 ('page_clerkperformance',),
 ('page_expensealongtime',),
 ('page_inventory',),
 ('page_pagecustom',),
 ('page_salealongtime',),
 ('page_salelabel',),
 ('page_saleperiod',),
 ('page_salepriceio',),
 ('test_user_clerk',),
 ('user_clerk',),
 ('user_customer',),
 ('user_userprofile',),
 ('user_userprofile_groups',),
 ('user_userprofile_user_permissions',),
 ('xadmin_boo

In [62]:
metadata = MetaData()
district_info = Table(
    'district_info',
    metadata,
    Column('id', Integer, Sequence('district_info_id_seq'), primary_key=True),
    Column('province', String(32)),
    Column('city', String(32)),
    Column('district', String(32)),
    Column('show_address', String(64)),
    Column('full_address', String(64)),
    Column('cnt_sum_certain', Integer)  # -1: 表示有确诊但是确诊人数不详
)
print(district_info.name)

district_info


In [63]:
import akshare as ak
covid_19_csse_daily_df = ak.covid_19_csse_daily(date="2020-04-06")
print(covid_19_csse_daily_df)

KeyboardInterrupt: 

NameError: name '__file__' is not defined